In [ ]:
import airgen
from airgen.utils.collect import data_collector
from typing import List, Tuple, Dict, Any, Optional, Callable

drones = [airgen.MultirotorClient()] *5

In [ ]:
num_drones = 5
for i in range(num_drones):
    drone_name = f"Drone{i}" if i>0 else "Drone"
    drones[i].enableApiControl(True, vehicle_name=drone_name)
    drones[i].takeoffAsync(vehicle_name=drone_name)

In [ ]:
drones[0].moveToPositionAsync(0, 10, -10, 5)

In [ ]:
from grid.model.perception.detection.gdino import GroundingDINO
import warnings
warnings.filterwarnings("ignore")

gdinomodel = GroundingDINO(use_local=True)
img = drones[0].getImages("front_center",[airgen.ImageType.Scene])[0][0]
boxes, scores, labels = gdinomodel.run(img, "substation .")
print(boxes, scores, labels)

In [ ]:
from grid.model.perception.vlm.llava import LLaVA
import warnings
warnings.filterwarnings("ignore")


llava = LLaVA(use_local = True)
outputs = llava.run(img, "What do you see?")
print(outputs)

In [ ]:
from grid.model.perception.depth.midas import MIDAS
import warnings
warnings.filterwarnings("ignore")

midas = MIDAS(use_local = True)
depth = midas.run(img)
print("Minimum depth value", depth.min(), "meters")
print("Maximum depth value", depth.max(), "meters")

In [ ]:
num_episodes = 1
import numpy as np

x_bounds = (20, 50)  
y_bounds = (30, 50)  
num_values = 15     


x_values = np.random.uniform(x_bounds[0], x_bounds[1], num_values)
y_values = np.random.uniform(y_bounds[0], y_bounds[1], num_values)

for i in range(num_episodes):
    for i in range(num_drones):
        drone_name = f"Drone{i}" if i>0 else "Drone"
        drones[i].moveToPositionAsync(x_values[i], y_values[i], -10, 5, vehicle_name=drone_name)

In [ ]:
def readSensors(client: airgen.MultirotorClient(), drone_name: 'str') -> dict:
    sensor_data = {}
    img = client.getImages("front_center",[airgen.ImageType.Scene], vehicle_name=drone_name)[0]
    sensor_data['rgb'] = img
    sensor_data['depth'] = midas.run(img[0])
    sensor_data['annot'] = gdinomodel.run(img[0], "windmill .")
    return sensor_data

In [ ]:
@data_collector(readSensors, time_delta=0.1)
def move_task(
    client: airgen.MultirotorClient, drone_name: 'str', position: Tuple[float], **kwargs
) -> None | Tuple[None, List[dict]]:
    client.moveToPositionAsync(position[0],position[1], position[2], position[3], vehicle_name=drone_name)

import numpy as np

x_bounds = (100, 50)  
y_bounds = (100, 50)  
num_values = 50     


x_values = np.random.uniform(x_bounds[0], x_bounds[1], num_values)
y_values = np.random.uniform(y_bounds[0], y_bounds[1], num_values)

for i in range(num_drones):
    drone_name = f"Drone{i}" if i>0 else "Drone"
    _, sensor_data = move_task(drones[i], drone_name, (x_values[i], y_values[i], -10, 5), _collect_data=True)
    
for i, data in enumerate(sensor_data):
    lidar = data["lidar"]
    rgb, _ = data["rgb"]
    rr.log("grid/imagery",rr.Image(rgb))
    rr.log("pointcloud", rr.Points3D(np.array(lidar.point_cloud).reshape(-1, 3)))

print(f"collected {len(sensor_data)} measurements during moving task")